<h1 style="font-weight: bold;">Titanic Survival</h1>

## 1) **Import libraries**
load the libraries to be used in the notebook

In [1]:
import os
import pandas as pd
import pprint
import numpy as np
import altair as alt
from typing import List, Any, Dict, Tuple, Literal
from sklearn.model_selection import train_test_split

In [2]:
try:
   from google.colab import drive
   drive.mount("content")
   is_running_on_colab = True
except ImportError:
   is_running_on_colab = False

## 2) **Setting notebook**

In [3]:

if not is_running_on_colab:
    alt.renderers.enable('mimetype')
else:
    alt.renderers.enable('html')


## 3) **Load the data**
<p>Load the csv file to pandas dataframe</p>

In [4]:
folder_path = None
if os.path.exists('./data/'):
    folder_path = './data/'
elif os.path.exists('content/MyDrive/Datasets/titanic_data'):
    folder_path = 'content/MyDrive/Datasets/titanic_data'
else:
    raise ValueError("Unable to find titanic train and test csv files")

train_path = os.path.join(folder_path, "train.csv")
test_path = os.path.join(folder_path, "test.csv")

In [5]:
# Set the PassengerId column as index column.
index_col = 'PassengerId'

# Load the train and test dataframe.
train_df = pd.read_csv(train_path, index_col=index_col)
test_df = pd.read_csv(test_path, index_col=index_col)

## 4) **Data statistics**
<p>Analyse train and test dataframe</p>

### i) Rows in train data
<p>View train dataframe top and bottom rows</p>

In [6]:
# First five rows in train data.
train_df.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [7]:
# First five rows in test data.
train_df.tail()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.00,NaN,S
888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.00,B42,S
889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.45,NaN,S
890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.00,C148,C
891,0,3,"Dooley, Mr. Patrick",male,32.0,0,0,370376,7.75,NaN,Q


### ii) Rows in test data
<p>View test dataframe top and bottom rows</p>

In [8]:
# First five rows in test data.
test_df.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,
892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [9]:
# Last five rows in test data
test_df.tail()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,
1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S
1309,3,"Peter, Master. Michael J",male,NaN,1,1,2668,22.3583,NaN,C


### iii) Data description
<p>Get the know the datasets</p>

#### a) Data shapes

In [10]:
# Train dataframe shape.
train_shape = train_df.shape
print(f"Train shape:\n\trows: {train_shape[0]}\n\tcolumns: {train_shape[1]}")

Train shape:
	rows: 891
	columns: 11


In [11]:
# Test dataframe shape.
test_shape = test_df.shape
print(f"Test shape:\n\trows: {test_shape[0]}\n\tcolumns: {test_shape[1]}")

Test shape:
	rows: 418
	columns: 10


#### b) Data column names

In [12]:
columns = train_df.columns.tolist()
pprint.pprint(columns)

['Survived',
 'Pclass',
 'Name',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Ticket',
 'Fare',
 'Cabin',
 'Embarked']


In [13]:
train_df.dtypes

Survived      int64
Pclass        int64
Name         object
Sex          object
Age         float64
SibSp         int64
Parch         int64
Ticket       object
Fare        float64
Cabin        object
Embarked     object
dtype: object

#### i) Converting train data frame low unique values

In [14]:
def convert_low_unique_value_to_str(data: pd.DataFrame) -> pd.DataFrame:
    """
    Convert low unique values columns to string.

    Args
    ----
    data: Dataframe to change low unique columns.

    :return: Dataframe.
    """
    # Change Survived, Pclass, SibSp and Parch to string data type for analysis.
    data_copy = data.copy()

    # Loop over the column name to turn to object
    for col in ("Survived", "Pclass", "SibSp", "Parch"):
        data_copy[col] = data_copy[col].astype(str)

    return data_copy


ana_train_data = convert_low_unique_value_to_str(train_df)
ana_train_data.dtypes

Survived     object
Pclass       object
Name         object
Sex          object
Age         float64
SibSp        object
Parch        object
Ticket       object
Fare        float64
Cabin        object
Embarked     object
dtype: object

#### c) Data information

In [15]:
# Train data information.
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 891 entries, 1 to 891
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Name      891 non-null    object 
 3   Sex       891 non-null    object 
 4   Age       714 non-null    float64
 5   SibSp     891 non-null    int64  
 6   Parch     891 non-null    int64  
 7   Ticket    891 non-null    object 
 8   Fare      891 non-null    float64
 9   Cabin     204 non-null    object 
 10  Embarked  889 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 83.5+ KB


In [16]:
# Test data information
test_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 418 entries, 892 to 1309
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pclass    418 non-null    int64  
 1   Name      418 non-null    object 
 2   Sex       418 non-null    object 
 3   Age       332 non-null    float64
 4   SibSp     418 non-null    int64  
 5   Parch     418 non-null    int64  
 6   Ticket    418 non-null    object 
 7   Fare      417 non-null    float64
 8   Cabin     91 non-null     object 
 9   Embarked  418 non-null    object 
dtypes: float64(2), int64(3), object(5)
memory usage: 35.9+ KB


#### d) Describing the data

In [17]:
# Train dataframe description
train_df.describe().T

,count,mean,std,min,25%,50%,75%,max
Survived,891.0,0.383838,0.486592,0.00,0.0000,0.0000,1.0,1.0000
Pclass,891.0,2.308642,0.836071,1.00,2.0000,3.0000,3.0,3.0000
Age,714.0,29.699118,14.526497,0.42,20.1250,28.0000,38.0,80.0000
SibSp,891.0,0.523008,1.102743,0.00,0.0000,0.0000,1.0,8.0000
Parch,891.0,0.381594,0.806057,0.00,0.0000,0.0000,0.0,6.0000
Fare,891.0,32.204208,49.693429,0.00,7.9104,14.4542,31.0,512.3292


<p style="font-weight: bold; font-style: italic;">Description of numerical columns in train data</p>

In [18]:

ana_train_data.describe().T

,count,mean,std,min,25%,50%,75%,max
Age,714.0,29.699118,14.526497,0.42,20.1250,28.0000,38.0,80.0000
Fare,891.0,32.204208,49.693429,0.00,7.9104,14.4542,31.0,512.3292


<p style="font-weight: bold; font-style: italic;">Description of categorical columns in train data</p>

In [19]:
ana_train_data.describe(exclude=float).T

,count,unique,top,freq
Survived,891,2,0,549
Pclass,891,3,3,491
Name,891,891,"Braund, Mr. Owen Harris",1
Sex,891,2,male,577
SibSp,891,7,0,608
Parch,891,7,0,678
Ticket,891,681,347082,7
Cabin,204,147,B96 B98,4
Embarked,889,3,S,644


## 5) **Data split**
<p>Separate target from the data and then split the data into train and validation</p>

In [20]:
# Target name.
target_name = "Survived"

# Separating target column from the feature columns.
X = train_df.copy()
y = X.pop(target_name).values.reshape(-1, 1)

# View the shape X and y.
print(f'''
X shape: {X.shape}
y shape: {y.shape}
''')


X shape: (891, 10)
y shape: (891, 1)



In [21]:
# 25 percent will be for test data.
test_size = 0.25

# 🪓 Splitting data into train and validation data.
train_x, valid_x, train_y, valid_y = train_test_split(X, y, test_size=test_size)

# View the shape of the data.
print(f'''
train_x shape: {train_x.shape}
train_y shape: {train_y.shape}
valid_x shape: {valid_x.shape}
valid_y shape: {valid_y.shape}
''')


train_x shape: (668, 10)
train_y shape: (668, 1)
valid_x shape: (223, 10)
valid_y shape: (223, 1)



## 6) **Data cleaning**
<p>Modifying or removing data that is inaccurate, duplicate, incomplete, incorrectly formatted, or corrupted</p>

### i) Check the missing values (NaN)
<p>Search for missing value in the dataframes</p>

In [22]:
def check_n_nan(data: pd.DataFrame, only_nan: bool=True) -> pd.DataFrame:
    """
    Represents the summary of missing data in the
    data frame.

    Parameters:
    ----------
    data: A data frame to check for missing data.

    return: A dataframe with number of missing value (NaN) and the percentage.
    """
    # Compute the total NaN for each column.
    total_nan = data.isna().sum()

    if only_nan:
        total_nan = total_nan[total_nan > 0]

    # Columns
    column_names = total_nan.index

    # Compute the percentage of missing data.
    percentage = (((total_nan / data.shape[0]) * 100)
                  .round(2)
                  .astype(str)
                  .map(lambda x: x + '%')
                 )

    return pd.DataFrame({
        'columns': column_names,
        'total_nan': total_nan,
        'percent_nan': percentage
    }).reset_index(drop=True)

In [23]:
def plot_nan(data: pd.DataFrame, name: str | None=None, only_nan: bool=True, use_percent=False) -> None:
    nan_df = check_n_nan(data, only_nan)
    nan_df['percent_nan'] = nan_df['percent_nan'].str.replace('%', '').astype(float).round().astype(int)

    radius = 5
    y_name = 'percent_nan' if use_percent else 'total_nan'
    startup_title = 'Percentage' if use_percent else 'Number'
    y_title = f'{startup_title} of missing values'
    y_name = alt.Y(y_name).title(y_title)
    name = name if name else 'a dataset'
    title = f'{startup_title} Of Missing values In {name.title()}'
    subtitle = f'A chart represents missing data in {name}'

    base = alt.Chart(
        nan_df,
        title=alt.Title(
            title,
            subtitle=subtitle,
            anchor='start',
        )
    ).mark_bar(
        cornerRadiusTopLeft=radius,
        cornerRadiusTopRight=radius
    ).encode(
        x = alt.X('columns').title('Columns names'),
        y = y_name,
        color = alt.Color('columns').scale(
            scheme='lightgreyred',
            reverse=True
        ).legend(orient='right'),
        tooltip = ["columns", "total_nan", 'percent_nan'],
        text = 'columns'
    ).configure_axis(
        grid=False
    ).configure_view(
        stroke=None
    )

    return base

#### a) Observing missing data in train and validation data

##### One) X train dataset

In [24]:
check_n_nan(train_x)

,columns,total_nan,percent_nan
0,Age,130,19.46%
1,Cabin,508,76.05%
2,Embarked,2,0.3%


In [25]:
plot_nan(
    train_x,
    name='x train dataset',
    only_nan=False
)

<VegaLite 5 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/display_frontends.html#troubleshooting


**conclusion:** X train data sample contains missing data of 21.11%(141) of Age column, 78.29%(523) of cabin column and 0.3%(2) of Embarked column are the amount of missing data in the dataset from the data size of 668.

##### Two) X validation dataset

In [26]:
check_n_nan(valid_x)

,columns,total_nan,percent_nan
0,Age,47,21.08%
1,Cabin,179,80.27%


In [27]:
plot_nan(
    valid_x,
    name='x validation dataset',
    only_nan=False,
    use_percent=True
)

<VegaLite 5 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/display_frontends.html#troubleshooting


**conclusion:** In X validation data sample there are 16.14%(36) of Age column and 73.54%(164) of cabin column are the amount of missing data in the dataset from the data size of 223.

#### b) Observing missing data in test dataset

In [28]:
check_n_nan(test_df)

,columns,total_nan,percent_nan
0,Age,86,20.57%
1,Fare,1,0.24%
2,Cabin,327,78.23%


In [29]:
plot_nan(
    test_df,
    name='test dataset',
    only_nan=False,
    use_percent=False
)

<VegaLite 5 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/display_frontends.html#troubleshooting


**conclusion:** The test data frame contains missing data of 20.57%(86) of Age column, 78.23%(327) of cabin column and 0.24%(1) of Fare column are the amount of missing data in the dataset from the data size of 418.

### ii) Handling the missing values (NaN)
<p>Filling missing value in the dataframes</p>

In [30]:
class FillNaN:
    def __init__(self, dfs: List[pd.DataFrame], fill_with: Dict[str, Any]) -> None:
        self.dfs = dfs
        self.fill_with = fill_with

    @property
    def fill(self) -> List[pd.DataFrame]:
        """
        Fills missing values in the data frames.
        :return: A tuple of data frames
        """
        lst = []
        for df in self.dfs:
            for col, strat in self.fill_with.items():
                df = self.strategy(df, strat=strat, col_name=col)
            lst.append(df)
        return lst

    @staticmethod
    def strategy(df: pd.DataFrame, strat: str, col_name: str) -> pd.DataFrame:
        match strat:
            case 'mean':
                df[col_name] = df[col_name].fillna(df[col_name].mean())
            case 'mode':
                df[col_name] = df[col_name].fillna(df[col_name].mode()[0])
            case 'min':
                df[col_name] = df[col_name].fillna(df[col_name].min())
            case 'max':
                df[col_name] = df[col_name].fillna(df[col_name].max())
            case 'median':
                df[col_name] = df[col_name].fillna(df[col_name].median())
            case 'bfill':
                df[col_name] = df[col_name].bfill()
            case 'ffill':
                df[col_name] = df[col_name].ffill()
            case "drop":
                df = df.drop(col_name, axis=1)
            case _:
                raise ValueError(f"Ooh, strategy has received invalid value `{strat}` for strat")
        return df

Filling the missing value in age with mean, Fare with ffill, Embarked with mode and Cabin with drop.

In [31]:
fill = FillNaN(
    [train_x, valid_x, test_df],
    {"Age": "mean",
     "Fare": "ffill",
     "Embarked": "mode",
     "Cabin": "drop"})

In [32]:
clean_train_x, clean_valid_x, clean_test_df = fill.fill

Check if there still any missing values in the data.

**Clean train x**

In [33]:
check_n_nan(clean_train_x, only_nan=False)

,columns,total_nan,percent_nan
0,Pclass,0,0.0%
1,Name,0,0.0%
2,Sex,0,0.0%
3,Age,0,0.0%
4,SibSp,0,0.0%
5,Parch,0,0.0%
6,Ticket,0,0.0%
7,Fare,0,0.0%
8,Embarked,0,0.0%


**Clean valid x**

In [34]:
check_n_nan(clean_valid_x, only_nan=False)

,columns,total_nan,percent_nan
0,Pclass,0,0.0%
1,Name,0,0.0%
2,Sex,0,0.0%
3,Age,0,0.0%
4,SibSp,0,0.0%
5,Parch,0,0.0%
6,Ticket,0,0.0%
7,Fare,0,0.0%
8,Embarked,0,0.0%


**Clean test df**

In [35]:
check_n_nan(clean_test_df, only_nan=False)

,columns,total_nan,percent_nan
0,Pclass,0,0.0%
1,Name,0,0.0%
2,Sex,0,0.0%
3,Age,0,0.0%
4,SibSp,0,0.0%
5,Parch,0,0.0%
6,Ticket,0,0.0%
7,Fare,0,0.0%
8,Embarked,0,0.0%


### iii) Handling the categorical columns
<p>Encode categorical column to integer and drop high cardinality columns in the dataframes</p>

In [36]:
class CategoryEncoder:
    def __init__(
            self,
            datasets: List[pd.DataFrame],
            handle_with: Dict[str, str],
            ) -> None:
        self.datasets = datasets
        self.handle_with = handle_with

    def strategy(self, df: pd.DataFrame, col: str, strat: str) -> pd.DataFrame:
        match strat:
            case 'label_encode':
                series = df[col].astype('category').cat.codes
                df[col] = series
            case 'one_hot_encode':
                data = pd.get_dummies(
                    df[col],
                    columns=list(df[col].unique())
                    ).astype(int)
                df = pd.concat([df, data], axis=1)
                df = df.drop(col, axis=1)
            case 'drop':
                df = df.drop(col, axis=1)
            case _:
                raise ValueError()

        return df

    def encode(self) -> List[pd.DataFrame]:
        """
        Encode categorical columns to integer.
        :return: A tuple of data frames
        """

        datasets = []

        for dataset in self.datasets:
            for col, strat in self.handle_with.items():
                dataset = self.strategy(dataset, col, strat)
            datasets.append(dataset)

        return datasets

In [37]:
# First five rows from the clean train x.
clean_train_x.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
PassengerId,,,,,,,,,
482,2,"Frost, Mr. Anthony Wood ""Archie""",male,29.857193,0,0,239854,0.0000,S
104,3,"Johansson, Mr. Gustaf Joel",male,33.000000,0,0,7540,8.6542,S
210,1,"Blank, Mr. Henry",male,40.000000,0,0,112277,31.0000,C
656,2,"Hickman, Mr. Leonard Mark",male,24.000000,2,0,S.O.C. 14879,73.5000,S
268,3,"Persson, Mr. Ernst Ulrik",male,25.000000,1,0,347083,7.7750,S


In [38]:
# Exclude the int and float from the data.
cat_df = clean_train_x.select_dtypes(exclude=[int, float])
cat_df.head()

,Name,Sex,Ticket,Embarked
PassengerId,,,,
482,"Frost, Mr. Anthony Wood ""Archie""",male,239854,S
104,"Johansson, Mr. Gustaf Joel",male,7540,S
210,"Blank, Mr. Henry",male,112277,C
656,"Hickman, Mr. Leonard Mark",male,S.O.C. 14879,S
268,"Persson, Mr. Ernst Ulrik",male,347083,S


In [39]:
# Fetch the categorical columns.
cat_columns = list(cat_df.columns)
cat_columns

['Name', 'Sex', 'Ticket', 'Embarked']

Check for columns with high cardinality column in the data.

In [40]:
clean_train_x[cat_columns].nunique()

Name        668
Sex           2
Ticket      533
Embarked      3
dtype: int64

In [41]:
# Instantiate the CategoryEncoder
encoder = CategoryEncoder(
    [clean_train_x, clean_valid_x, clean_test_df],
    {
        "Sex": "label_encode",
        "Embarked": "one_hot_encode",
        "Name": "drop",
        "Ticket": "drop"
    }
)

# Encode the column
num_train_x, num_valid_x, num_test_df = encoder.encode()

In [42]:
num_train_x.dtypes

Pclass      int64
Sex          int8
Age       float64
SibSp       int64
Parch       int64
Fare      float64
C           int64
Q           int64
S           int64
dtype: object

## 7) **Data visualisation**
<p>Represeting the data in graphical way like making chart</p>

In [43]:
visual_df = pd.concat([clean_train_x, clean_valid_x])
visual_df['Survived'] = np.concatenate([train_y, valid_y])
visual_df.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,Survived
PassengerId,,,,,,,,,,
482,2,"Frost, Mr. Anthony Wood ""Archie""",1,29.857193,0,0,239854,0.0000,S,0
104,3,"Johansson, Mr. Gustaf Joel",1,33.000000,0,0,7540,8.6542,S,0
210,1,"Blank, Mr. Henry",1,40.000000,0,0,112277,31.0000,C,1
656,2,"Hickman, Mr. Leonard Mark",1,24.000000,2,0,S.O.C. 14879,73.5000,S,0
268,3,"Persson, Mr. Ernst Ulrik",1,25.000000,1,0,347083,7.7750,S,1


In [ ]:
class Graph:
    def __init__(self, data: pd.DataFrame) -> None:
        self.__data = data

    def value_mapping(
        self, 
        x: str, 
        map_values: Dict[str, Any] | None, 
        color: List[str] | str) -> Tuple[pd.DataFrame, alt.Color]:
        
        # Make copy then change column data to str
        data = self.__data.copy()
        data[x] = data[x].astype(str)
        if map_values:
            data[x] = data[x].map(lambda x: map_values[x])

        # Color map.
        if isinstance(color, list):
            alt_color = (
                alt.Color(x).scale(range=color, reverse=True).legend(orient="right")
            )
        else:
            alt_color = (
                alt.Color(x).scale(scheme=color, reverse=True).legend(orient="right")
            )
        return (data, alt_color)

    def bar_chart(
        self,
        x: str,
        map_values: Dict[str, Any] | None = None,
        color: List | str = "paired",
        width: int = 300,
        height: int = 400,
        x_title: str | None = None,
        y_title: str | None = None,
    ) -> alt.ChartDataType:# Make copy then change column data to str

        data, alt_color = self.value_mapping(
            x,
            map_values=map_values,
            color=color
        )

        x_title = x_title.title() if x_title else x.title()
        y_title = y_title.title() if y_title else "Number of Passengers"

        # Return the alt chart
        bar_layer = (
            alt.Chart(data)
            .mark_bar(cornerRadius=10)
            .encode(
                x=alt.X(
                    x, title=x_title, axis=alt.Axis(grid=False, labelFontWeight="bold")
                ),
                y=alt.Y(
                    "count()",
                    title=y_title,
                    axis=alt.Axis(grid=False, labelFontWeight="bold"),
                ),
                color=alt_color.legend(orient="right"),
                tooltip=[
                    alt.Tooltip(x, title=x_title),
                    alt.Tooltip("count()", title=y_title),
                ],
            )
            .properties(
                width=width,
                height=height,
                title=alt.Title(
                    f"{x.title()} Distribution",
                    fontSize=18,
                    subtitle="Source: Titanic",
                    subtitleFontSize=12,
                    dy=-10,
                ),
            )
        )

        text_layer = (
            alt.Chart(data)
            .mark_text(
                align="center",
                baseline="bottom",  # Align text to the bottom of its bounding box
                dy=-5,  # Adjust vertical position (negative for above the bar)
                fontSize=12,
                fontWeight="bold",
            )
            .encode(x=x, y="count()", text=alt.Text("count()"))
        )

        return bar_layer + text_layer  # .configure_view(

    def donut_chart(
        self, 
        x: str,
        map_values: Dict[str, Any] | None = None,
        color: List | str = "paired",
    ) -> alt.ChartDataType:

        data, alt_color = self.value_mapping(
            x,
            map_values=map_values,
            color=color
        )

        value_counts = data[x].value_counts().reset_index()
        value_counts["count"] = (value_counts["count"] / data.shape[0]).round(3)
        
        donut = alt.Chart(value_counts).mark_arc(innerRadius=50).encode(
            theta="count",
            color=f"{x}:N",
            tooltip=[
                alt.Tooltip(f"{x}".title()),
                alt.Tooltip('count', title='Percentage')
            ]
        )
        return donut

    def concat(self, *args, use: Literal["vconcat", "hconcat"] = "vconcat"):
        if use == "vconcat":
            return alt.vconcat(*args).configure_view(stroke=None)
        elif use == "hconcat":
            return alt.hconcat(*args).configure_view(stroke=None)
        else:
            raise ValueError("Received invalid value `%s`" % use)


graph = Graph(clean_train_x)
bar = graph.bar_chart("Embarked")
donut = graph.donut_chart("Embarked")

graph.concat(bar, donut, use="hconcat")

<VegaLite 5 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/display_frontends.html#troubleshooting


In [45]:
x = clean_train_x['Embarked'].value_counts().reset_index()
x["count"] = (x["count"] / clean_train_x.shape[0]).round(3)

In [46]:
x

,Embarked,count
0,S,0.722
1,C,0.187
2,Q,0.091
